In [1]:
# Violence Detection System with PyTorch Hub YOLOv5 and ONNX
# This version uses torch.hub.load for reliable YOLOv5 integration

# === CONFIGURATION - CHANGE THESE PATHS TO YOUR MODEL LOCATIONS ===
VIOLENCE_MODEL_PATH = "/content/drive/MyDrive/violence_detector_model (2).onnx"  # Path to violence detector

# === INSTALLATION - AUTOMATICALLY INSTALLS REQUIRED PACKAGES ===
# Install required packages
!pip install -q torch torchvision onnxruntime opencv-python gradio tqdm

# === IMPORTS - ALL REQUIRED LIBRARIES ===
import os
import sys
import cv2
import torch
import numpy as np
import time
import onnxruntime as ort
from collections import deque
import gradio as gr
from tqdm.notebook import tqdm
from google.colab import drive
from datetime import datetime
import tempfile
import shutil
import torch.nn.functional as F

# === MOUNT GOOGLE DRIVE - AUTOMATICALLY MOUNTS TO ACCESS YOUR MODELS ===
drive.mount('/content/drive')

# === DEVICE SETUP - AUTOMATICALLY USES GPU IF AVAILABLE ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Clear GPU memory if using CUDA
if device.type == 'cuda':
    torch.cuda.empty_cache()

# === YOLOV5 PERSON DETECTOR USING PYTORCH HUB ===
class YOLOv5PersonDetector:
    """Person detector using YOLOv5 from PyTorch Hub"""

    def __init__(self, confidence_threshold=0.3):
        """Initialize YOLOv5 model from PyTorch Hub"""
        self.confidence_threshold = confidence_threshold
        self.device = device

        try:
            # Load YOLOv5 from PyTorch Hub
            print("Loading YOLOv5 from PyTorch Hub...")
            self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

            # Configure model settings
            self.model.conf = self.confidence_threshold  # Detection confidence threshold
            self.model.iou = 0.45  # NMS IoU threshold
            self.model.classes = [0]  # Only detect people (class 0 in COCO)
            self.model.max_det = 10  # Maximum number of detections per image

            # Move model to the appropriate device
            self.model.to(self.device)

            print("YOLOv5 model loaded successfully")
            self.model_loaded = True
        except Exception as e:
            print(f"Error loading YOLOv5 model: {e}")
            import traceback
            traceback.print_exc()
            print("Will use fallback detection method")
            self.model_loaded = False

    def detect(self, frame):
        """Detect people in the frame using YOLOv5"""
        try:
            if not self.model_loaded:
                # If model failed to load, use fallback
                return self.fallback_detect(frame)

            # Run YOLOv5 inference
            results = self.model(frame)

            # Extract person detections (should already be filtered to class 0)
            boxes = []

            # Parse results - this uses the YOLOv5 results format
            detections = results.xyxy[0].cpu().numpy()  # Get detection boxes in xyxy format

            for detection in detections:
                x1, y1, x2, y2, conf, cls = detection
                # Double-check it's a person (class 0) even though we filtered in the model
                if int(cls) == 0 and conf > self.confidence_threshold:
                    boxes.append((int(x1), int(y1), int(x2), int(y2), float(conf)))

            # If no people detected, use whole frame
            if not boxes:
                height, width = frame.shape[:2]
                boxes.append((0, 0, width, height, 1.0))

            return boxes
        except Exception as e:
            print(f"Error in YOLOv5 person detection: {e}")
            import traceback
            traceback.print_exc()
            # Fall back to simpler detection if YOLOv5 fails
            return self.fallback_detect(frame)

    def fallback_detect(self, frame):
        """Fallback detection method using OpenCV's HOG detector"""
        try:
            # Initialize HOG detector
            hog = cv2.HOGDescriptor()
            hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

            # Detect people
            boxes, weights = hog.detectMultiScale(
                frame,
                winStride=(8, 8),
                padding=(4, 4),
                scale=1.05
            )

            # Format detections
            result = []
            for (x, y, w, h), weight in zip(boxes, weights):
                result.append((x, y, x + w, y + h, weight))

            # If no people detected, use whole frame
            if not result:
                height, width = frame.shape[:2]
                result.append((0, 0, width, height, 1.0))

            return result
        except Exception as e:
            print(f"Error in fallback detection: {e}")
            # Return whole frame as ultimate fallback
            height, width = frame.shape[:2]
            return [(0, 0, width, height, 1.0)]

# === VIOLENCE DETECTION SYSTEM ===
class ViolenceDetectionSystem:
    """Violence detection system combining YOLOv5 and ONNX violence detector"""

    def __init__(self, violence_path, confidence_threshold=0.3):
        self.confidence_threshold = confidence_threshold
        self.device = device

        # Initialize YOLOv5 person detector
        self.person_detector = YOLOv5PersonDetector(confidence_threshold=0.3)

        # Load violence model
        self.load_violence_model(violence_path)

        # Initialize frame buffer for temporal processing
        self.frame_buffer = deque(maxlen=8)
        self.processing_times = deque(maxlen=30)
        self.detections_history = deque(maxlen=10)

        # For event tracking
        self.violence_events = []
        self.current_violence_event = None

        print(f"Violence detection system initialized on {self.device}")
        print(f"Violence confidence threshold: {self.confidence_threshold}")

    def load_violence_model(self, violence_path):
        """Load violence detection model with error handling"""
        try:
            # Load violence detection model (ONNX)
            print(f"Loading violence detection model from {violence_path}...")
            if os.path.exists(violence_path):
                # Setup ONNX runtime session
                providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if self.device.type == "cuda" else ['CPUExecutionProvider']
                self.violence_session = ort.InferenceSession(violence_path, providers=providers)
                self.input_name = self.violence_session.get_inputs()[0].name
                self.violence_model_loaded = True
                print("Violence detection model loaded successfully")

                # Print model input details for debugging
                print("Model Input Details:")
                for input in self.violence_session.get_inputs():
                    print(f"  Name: {input.name}, Shape: {input.shape}, Type: {input.type}")
            else:
                print(f"Warning: Violence detection model not found at {violence_path}.")
                print("Will run in person-detection-only mode")
                self.violence_session = None
                self.violence_model_loaded = False
        except Exception as e:
            print(f"Error loading violence model: {e}")
            import traceback
            traceback.print_exc()
            self.violence_session = None
            self.violence_model_loaded = False

    def preprocess_clip(self, frames):
        """Preprocess frames for violence detection"""
        try:
            # Resize frames
            resized_frames = [cv2.resize(frame, (224, 224)) for frame in frames]

            # Convert to normalized array
            np_frames = np.array(resized_frames, dtype=np.float32) / 255.0

            # Transpose from [T, H, W, C] to [C, T, H, W] for 3D CNN
            np_frames = np.transpose(np_frames, (3, 0, 1, 2))

            # Add batch dimension
            np_frames = np.expand_dims(np_frames, axis=0)

            return np_frames
        except Exception as e:
            print(f"Error preprocessing clip: {e}")
            # Return empty tensor as fallback
            return np.zeros((1, 3, 8, 224, 224), dtype=np.float32)

    def detect_violence(self, clip):
        """Detect violence in clip using the ONNX model"""
        if not self.violence_model_loaded:
            return False, 0.0

        try:
            # Run ONNX inference
            ort_inputs = {self.input_name: clip}
            ort_outputs = self.violence_session.run(None, ort_inputs)
            scores = ort_outputs[0][0]

            # Convert to probabilities
            probabilities = F.softmax(torch.tensor(scores), dim=0)

            # Get violence probability (assuming class 1 is violence)
            violence_prob = probabilities[1].item()

            # Check against threshold
            is_violent = violence_prob > self.confidence_threshold

            # Print higher probabilities for debugging
            if violence_prob > 0.1:
                print(f"Violence probability: {violence_prob:.4f} (Threshold: {self.confidence_threshold})")

            return is_violent, violence_prob
        except Exception as e:
            print(f"Error in violence detection: {e}")
            return False, 0.0

    def process_frame(self, frame, frame_idx=0, timestamp=None):
        """Process a single frame for violence detection"""
        start_time = time.time()

        # Make a copy for visualization
        vis_frame = frame.copy()

        # Add frame to buffer
        self.frame_buffer.append(frame)

        # Skip complete processing if buffer isn't full yet
        if len(self.frame_buffer) < 8:
            # Display that we're collecting frames
            cv2.putText(vis_frame, f"Collecting frames... ({len(self.frame_buffer)}/8)",
                      (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            processing_time = time.time() - start_time
            self.processing_times.append(processing_time)

            return vis_frame, False, 0.0

        # Detect people using YOLOv5
        person_boxes = self.person_detector.detect(frame)

        # Track violence detections for this frame
        frame_has_violence = False
        max_violence_prob = 0.0

        # Process each person region
        for box in person_boxes:
            x1, y1, x2, y2, confidence = box

            # Extract person region from each frame in buffer
            person_clips = []
            for buffered_frame in self.frame_buffer:
                # Ensure valid coordinates
                valid_x1 = max(0, x1)
                valid_y1 = max(0, y1)
                valid_x2 = min(buffered_frame.shape[1], x2)
                valid_y2 = min(buffered_frame.shape[0], y2)

                if valid_x2 <= valid_x1 or valid_y2 <= valid_y1:
                    # Invalid region, use whole frame
                    region = buffered_frame
                else:
                    # Extract valid region
                    region = buffered_frame[valid_y1:valid_y2, valid_x1:valid_x2]

                person_clips.append(region)

            # Preprocess clip
            clip = self.preprocess_clip(person_clips)

            # Detect violence
            is_violent, violence_prob = self.detect_violence(clip)

            # Track maximum violence probability
            max_violence_prob = max(max_violence_prob, violence_prob)

            # Update frame violence status
            if is_violent:
                frame_has_violence = True

            # Draw bounding box
            color = (0, 0, 255) if is_violent else (0, 255, 0)
            cv2.rectangle(vis_frame, (x1, y1), (x2, y2), color, 2)

            # Add label
            if self.violence_model_loaded:
                label = f"Violent: {violence_prob:.2f}" if is_violent else f"Normal: {1-violence_prob:.2f}"
                cv2.putText(vis_frame, label, (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            else:
                label = f"Person: {confidence:.2f}"
                cv2.putText(vis_frame, label, (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        if self.violence_model_loaded:
            # Update detection history
            self.detections_history.append(frame_has_violence)

            # Apply temporal smoothing - violence is detected if 30% or more recent frames show violence
            smooth_violence = sum(self.detections_history) >= len(self.detections_history) * 0.3

            # Track violence events
            if smooth_violence:
                if self.current_violence_event is None:
                    # Start new violence event
                    event_time = timestamp if timestamp else datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    self.current_violence_event = {
                        "start_frame": frame_idx,
                        "start_time": event_time,
                        "end_frame": frame_idx,
                        "end_time": event_time,
                        "max_probability": max_violence_prob
                    }
                else:
                    # Update existing violence event
                    self.current_violence_event["end_frame"] = frame_idx
                    self.current_violence_event["max_probability"] = max(
                        self.current_violence_event["max_probability"], max_violence_prob
                    )
                    if timestamp:
                        self.current_violence_event["end_time"] = timestamp
                    else:
                        self.current_violence_event["end_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            elif self.current_violence_event is not None:
                # End violence event
                self.violence_events.append(self.current_violence_event)
                self.current_violence_event = None

            # Add violence alert
            if smooth_violence:
                cv2.putText(vis_frame, "VIOLENCE DETECTED!", (vis_frame.shape[1]//2 - 150, 50),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                cv2.rectangle(vis_frame, (0, 0), (vis_frame.shape[1], vis_frame.shape[0]), (0, 0, 255), 5)
        else:
            # No violence model, just display person detection
            smooth_violence = False

        # Calculate processing time and FPS
        processing_time = time.time() - start_time
        self.processing_times.append(processing_time)
        avg_time = sum(self.processing_times) / len(self.processing_times)
        fps = 1.0 / avg_time if avg_time > 0 else 0

        # Display FPS and processing time
        cv2.putText(vis_frame, f"FPS: {fps:.1f}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        return vis_frame, smooth_violence, max_violence_prob

    def process_video(self, video_path, output_path=None, skip_frames=1):
        """Process a video file for violence detection"""
        try:
            # Open video
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Error: Could not open video {video_path}")
                return None, {"error": "Could not open video file"}

            # Get video properties
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            print(f"Processing video: {video_path}")
            print(f"Resolution: {width}x{height}, FPS: {fps}, Total frames: {frame_count}")

            # Create output video if requested
            if output_path:
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

            # Reset state
            self.frame_buffer.clear()
            self.detections_history.clear()
            self.violence_events = []
            self.current_violence_event = None

            # Initialize frame counter
            frame_idx = 0
            processed_frames = 0

            # Track violence stats
            violence_detected_frames = 0

            # Create progress bar
            pbar = tqdm(total=frame_count, desc="Processing video")

            while cap.isOpened():
                try:
                    ret, frame = cap.read()
                    if not ret:
                        break

                    # Process every N frames to improve speed
                    if frame_idx % skip_frames == 0:
                        # Calculate timestamp
                        timestamp = f"{frame_idx/fps:.2f}s"

                        try:
                            # Process frame with error handling
                            processed_frame, is_violent, _ = self.process_frame(frame, frame_idx, timestamp)

                            # Track stats
                            if is_violent:
                                violence_detected_frames += 1

                            # Write to output video if requested
                            if output_path:
                                out.write(processed_frame)
                        except Exception as e:
                            print(f"Error processing frame {frame_idx}: {e}")
                            # Write original frame to output if processing fails
                            if output_path:
                                out.write(frame)

                        processed_frames += 1

                    frame_idx += 1
                    pbar.update(1)
                except Exception as e:
                    print(f"Error in frame loop: {e}")
                    frame_idx += 1
                    pbar.update(1)

            # Finalize last violence event if exists
            if self.current_violence_event is not None:
                self.violence_events.append(self.current_violence_event)

            # Release resources
            cap.release()
            if output_path:
                out.release()
            pbar.close()

            # Calculate statistics
            violence_percentage = (violence_detected_frames / processed_frames) * 100 if processed_frames > 0 else 0

            # Return summary
            summary = {
                "total_frames": processed_frames,
                "violence_frames": violence_detected_frames,
                "violence_percentage": violence_percentage,
                "events": self.violence_events
            }

            print(f"\nDetected {len(self.violence_events)} violence events")
            print(f"Violence percentage: {violence_percentage:.2f}%")

            return output_path, summary

        except Exception as e:
            print(f"Error processing video: {e}")
            import traceback
            traceback.print_exc()
            return None, {"error": str(e)}

# === WEB INTERFACE FUNCTIONS ===
def process_video_for_gradio(video_file):
    """Process video for Gradio interface"""
    if video_file is None:
        return None, "No video uploaded. Please upload a video file."

    try:
        # Create a temporary directory for output
        temp_dir = tempfile.mkdtemp()
        output_path = os.path.join(temp_dir, "processed_video.mp4")

        # Process the video
        result_path, summary = detector.process_video(
            video_path=video_file,
            output_path=output_path,
            skip_frames=2  # Process every other frame for speed
        )

        # Handle errors
        if isinstance(summary, dict) and "error" in summary:
            return None, f"Error: {summary['error']}"

        # Format the summary
        if not os.path.exists(output_path):
            return None, "Error processing video: Output file not created."

        # Create a markdown report
        report = f"""
        ## Violence Detection Results

        ### Statistics
        - Total frames analyzed: {summary['total_frames']}
        - Frames with violence: {summary['violence_frames']} ({summary['violence_percentage']:.2f}%)

        ### Assessment
        {generate_assessment(summary['violence_percentage'])}

        ### Detected Events
        """

        if summary['events']:
            for i, event in enumerate(summary['events']):
                report += f"""
                **Event {i+1}**:
                - Start: {event['start_time']}
                - End: {event['end_time']}
                - Max Probability: {event['max_probability']:.2f}
                """
        else:
            report += "No specific violence events detected."

        return output_path, report

    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        print(f"Error in interface: {e}")
        print(error_details)
        return None, f"Error processing video: {str(e)}"

def generate_assessment(violence_percentage):
    """Generate assessment text based on violence percentage"""
    if violence_percentage >= 60:
        return "⚠️ **HIGH VIOLENCE CONTENT DETECTED**\nThis video contains significant violent behavior."
    elif violence_percentage >= 40:
        return "⚠️ **MEDIUM VIOLENCE CONTENT DETECTED**\nThis video contains some violent behavior."
    elif violence_percentage > 20:
        return "⚠️ **LOW VIOLENCE CONTENT DETECTED**\nThis video contains minimal violent behavior."
    else:
        return "✅ **NO VIOLENCE DETECTED**\nNo violent behavior was detected in this video."

# === MAIN EXECUTION ===
# Initialize the system with a lower confidence threshold
detector = ViolenceDetectionSystem(
    violence_path=VIOLENCE_MODEL_PATH,
    confidence_threshold=0.6  # Lower threshold to detect more violence
)

# Create and launch the Gradio interface
interface = gr.Interface(
    fn=process_video_for_gradio,
    inputs=gr.Video(label="Upload Video for Violence Detection"),
    outputs=[
        gr.Video(label="Processed Video"),
        gr.Markdown(label="Analysis Results")
    ],
    title="Violence Detection System",
    description="""
    This system combines YOLOv5 person detection with a specialized violence detector.
    Upload a video to detect violent behavior and receive a detailed analysis.

    Processing may take a few minutes depending on video length and resolution.
    """,
    examples=[],
    flagging_mode="never",
    theme=gr.themes.Soft()
)

# Launch with public sharing enabled
interface.launch(share=True, debug=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-4-28 Python-3.11.12 torch-2.6.0+cu124 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 125MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


YOLOv5 model loaded successfully
Loading violence detection model from /content/drive/MyDrive/violence_detector_model (2).onnx...
Violence detection model loaded successfully
Model Input Details:
  Name: input, Shape: ['batch_size', 3, 8, 224, 224], Type: tensor(float)
Violence detection system initialized on cpu
Violence confidence threshold: 0.6
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4cac508f0febe65cdc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


...

Violence probability: 0.1663 (Threshold: 0.6)
Violence probability: 0.3160 (Threshold: 0.6)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Violence probability: 0.1421 (Threshold: 0.6)
Violence probability: 0.8118 (Threshold: 0.6)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Violence probability: 0.1313 (Threshold: 0.6)
Violence probability: 0.8688 (Threshold: 0.6)

Detected 7 violence events
Violence percentage: 52.26%
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4cac508f0febe65cdc.gradio.live


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# === ALERT NOTIFICATION SYSTEM ===
# Install required packages
!pip install -q python-telegram-bot==13.7

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import asyncio
import telegram
import threading
import queue
import time

class AlertSystem:
    """Alert system for violence detection notifications"""

    def __init__(self):
        # Initialize notification queues
        self.email_queue = queue.Queue()
        self.telegram_queue = queue.Queue()

        # Start notification threads
        self.start_notification_threads()

        print("Alert system initialized")

    def start_notification_threads(self):
        """Start background threads for notifications"""
        # Email notification thread
        email_thread = threading.Thread(
            target=self._process_email_queue,
            daemon=True
        )
        email_thread.start()

        # Telegram notification thread
        telegram_thread = threading.Thread(
            target=self._process_telegram_queue,
            daemon=True
        )
        telegram_thread.start()

    def configure_email(self, sender_email, sender_password, recipient_email):
        """Configure email settings"""
        self.sender_email = sender_email
        self.sender_password = sender_password
        self.recipient_email = recipient_email
        self.email_configured = True
        print(f"Email alerts configured for {recipient_email}")

    def configure_telegram(self, bot_token, chat_id):
        """Configure Telegram settings"""
        self.telegram_bot_token = bot_token
        self.telegram_chat_id = chat_id
        self.telegram_configured = True
        print(f"Telegram alerts configured for chat ID {chat_id}")

    def send_alert(self, message, image=None):
        """Queue alerts for delivery through configured channels"""
        print(f"Alert triggered: {message}")

        # Queue email notification if configured
        if hasattr(self, 'email_configured') and self.email_configured:
            self.email_queue.put({
                'message': message,
                'image': image
            })

        # Queue telegram notification if configured
        if hasattr(self, 'telegram_configured') and self.telegram_configured:
            self.telegram_queue.put({
                'message': message,
                'image': image
            })

    def _process_email_queue(self):
        """Process email notification queue in background"""
        while True:
            try:
                # Get notification from queue
                notification = self.email_queue.get(block=True, timeout=1.0)

                # Send email
                self._send_email_notification(
                    notification['message'],
                    notification['image']
                )

                # Mark task as done
                self.email_queue.task_done()

            except queue.Empty:
                # No notifications in queue
                time.sleep(1.0)
            except Exception as e:
                print(f"Error processing email notification: {e}")
                time.sleep(5.0)  # Delay before retrying

    def _process_telegram_queue(self):
        """Process telegram notification queue in background"""
        while True:
            try:
                # Get notification from queue
                notification = self.telegram_queue.get(block=True, timeout=1.0)

                # Send telegram
                asyncio.run(self._send_telegram_notification(
                    notification['message'],
                    notification['image']
                ))

                # Mark task as done
                self.telegram_queue.task_done()

            except queue.Empty:
                # No notifications in queue
                time.sleep(1.0)
            except Exception as e:
                print(f"Error processing telegram notification: {e}")
                time.sleep(5.0)  # Delay before retrying

    def _send_email_notification(self, message_text, image=None):
        """Send email notification"""
        try:
            # Create message container
            msg = MIMEMultipart()
            msg['From'] = self.sender_email
            msg['To'] = self.recipient_email
            msg['Subject'] = "ALERT: Violence Detected!"

            # Add message body
            body = MIMEText(message_text, 'plain')
            msg.attach(body)

            # Connect to Gmail SMTP server
            with smtplib.SMTP('smtp.gmail.com', 587) as server:
                server.starttls()  # Secure the connection
                server.login(self.sender_email, self.sender_password)
                server.send_message(msg)

            print(f"Email alert sent to {self.recipient_email}")
        except Exception as e:
            print(f"Failed to send email alert: {e}")

    async def _send_telegram_notification(self, message_text, image=None):
        """Send telegram notification asynchronously"""
        try:
            # Initialize telegram bot
            bot = telegram.Bot(token=self.telegram_bot_token)

            # Send text message
            await bot.send_message(
                chat_id=self.telegram_chat_id,
                text=f"🚨 VIOLENCE ALERT 🚨\n\n{message_text}"
            )

            # Send image if provided
            if image is not None:
                # Save image to temporary file
                temp_image_path = "/tmp/violence_alert.jpg"
                cv2.imwrite(temp_image_path, image)

                # Send image file
                with open(temp_image_path, 'rb') as image_file:
                    await bot.send_photo(
                        chat_id=self.telegram_chat_id,
                        photo=image_file,
                        caption="Violence detected frame"
                    )

            print(f"Telegram alert sent to chat ID {self.telegram_chat_id}")
        except Exception as e:
            print(f"Failed to send telegram alert: {e}")

# === SETUP INSTRUCTIONS (GMAIL) ===
# 1. Gmail requires "Less secure app access" or an "App Password"
# 2. Go to your Google Account > Security
# 3. For App Passwords: Enable 2FA, then create an App Password for "Mail"
# 4. Use your Gmail address and the App Password in the configure_email method

# === SETUP INSTRUCTIONS (TELEGRAM) ===
# 1. Create a Telegram bot via BotFather (https://t.me/botfather)
# 2. Save the provided bot token
# 3. Start a chat with your bot
# 4. Get your chat ID using the @userinfobot
# 5. Use the bot token and chat ID in the configure_telegram method

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.1/490.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.9 MB/s eta 0:00:00


In [3]:
# === ENHANCED VIOLENCE DETECTION SYSTEM WITH ALERTS ===

# First, create the alert system instance
alert_system = AlertSystem()

# Modify the ViolenceDetectionSystem class to integrate alerts
class ViolenceDetectionSystemWithAlerts(ViolenceDetectionSystem):
    """Enhanced violence detection system with alert capabilities"""

    def __init__(self, violence_path, confidence_threshold=0.3, alert_system=None):
        # Initialize parent class
        super().__init__(violence_path, confidence_threshold)

        # Set alert system
        self.alert_system = alert_system

        # Track alert status to avoid duplicate alerts
        self.alert_sent = False
        self.alert_cooldown = 60  # Seconds between alerts
        self.last_alert_time = 0

        print("Violence detection system with alerts initialized")

    def process_frame(self, frame, frame_idx=0, timestamp=None):
        """Process a single frame for violence detection with alert capability"""
        # Process frame using parent method
        vis_frame, is_violent, violence_prob = super().process_frame(frame, frame_idx, timestamp)

        # Check if we should send an alert
        current_time = time.time()
        if (is_violent and
            self.alert_system is not None and
            not self.alert_sent and
            current_time - self.last_alert_time > self.alert_cooldown):

            # Create alert message
            if timestamp:
                time_info = f"at timestamp {timestamp}"
            else:
                time_info = f"at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

            message = (
                f"Violence detected {time_info} with {violence_prob:.2f} probability.\n"
                f"This requires your immediate attention."
            )

            # Send alert with current frame
            self.alert_system.send_alert(message, frame)

            # Update alert status
            self.alert_sent = True
            self.last_alert_time = current_time

            # Add alert indicator to frame
            cv2.putText(
                vis_frame,
                "ALERT SENT!",
                (vis_frame.shape[1]//2 - 80, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2
            )

        # Reset alert status if no violence detected
        if not is_violent:
            self.alert_sent = False

        return vis_frame, is_violent, violence_prob

    def process_video(self, video_path, output_path=None, skip_frames=1):
        """Process a video file with alert capability"""
        # Reset alert status
        self.alert_sent = False
        self.last_alert_time = 0

        # Process video using parent method
        return super().process_video(video_path, output_path, skip_frames)

# Create a setup function for the Gradio interface
def setup_alerts(sender_email, sender_password, recipient_email, bot_token, chat_id):
    """Configure alert system from Gradio interface"""
    try:
        # Configure email if all fields provided
        if sender_email and sender_password and recipient_email:
            alert_system.configure_email(
                sender_email=sender_email,
                sender_password=sender_password,
                recipient_email=recipient_email
            )

        # Configure telegram if all fields provided
        if bot_token and chat_id:
            alert_system.configure_telegram(
                bot_token=bot_token,
                chat_id=chat_id
            )

        return "Alert system configured successfully!"
    except Exception as e:
        return f"Error configuring alert system: {str(e)}"

# Function to test alerts
def test_alert(message):
    """Test the alert system"""
    try:
        alert_system.send_alert(f"TEST ALERT: {message}")
        return "Test alert sent successfully!"
    except Exception as e:
        return f"Error sending test alert: {str(e)}"

Alert system initialized


In [4]:
# === ENHANCED GRADIO INTERFACE WITH ALERT CONFIGURATION ===

# Create Gradio tabs for different functionalities
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# Violence Detection System with Alerts")

    # Create tabs
    with gr.Tabs():
        # Tab 1: Video Processing
        with gr.TabItem("Process Video"):
            gr.Markdown("""
            ## Video Analysis
            Upload a video to detect violent behavior. The system will analyze the content and send alerts if configured.
            """)

            # Video input and output
            with gr.Row():
                video_input = gr.Video(label="Upload Video for Violence Detection")

            # Process button
            process_btn = gr.Button("Process Video")

            # Outputs
            with gr.Row():
                video_output = gr.Video(label="Processed Video")

            with gr.Row():
                results_md = gr.Markdown(label="Analysis Results")

            # Connect to processing function
            process_btn.click(
                fn=process_video_for_gradio,
                inputs=[video_input],
                outputs=[video_output, results_md]
            )

        # Tab 2: Alert Configuration
        with gr.TabItem("Configure Alerts"):
            gr.Markdown("""
            ## Alert Configuration
            Configure email and/or Telegram notifications for when violence is detected.

            ### Email Setup Instructions:
            1. For Gmail, use an App Password:
               - Enable 2FA on your Google account
               - Go to Google Account > Security > App passwords
               - Create a new App password for "Mail"
            2. Enter your Gmail address and the App password below

            ### Telegram Setup Instructions:
            1. Create a Telegram bot using BotFather (https://t.me/botfather)
            2. Save the bot token provided by BotFather
            3. Start a chat with your bot
            4. Get your chat ID by messaging @userinfobot on Telegram
            5. Enter both values below
            """)

            # Email configuration
            with gr.Group():
                gr.Markdown("### Email Configuration")
                email_sender = gr.Textbox(label="Sender Email (Gmail)")
                email_password = gr.Textbox(label="App Password", type="password")
                email_recipient = gr.Textbox(label="Recipient Email")

            # Telegram configuration
            with gr.Group():
                gr.Markdown("### Telegram Configuration")
                telegram_token = gr.Textbox(label="Bot Token")
                telegram_chat_id = gr.Textbox(label="Chat ID")

            # Configure button
            config_btn = gr.Button("Save Configuration")
            config_result = gr.Markdown()

            # Connect to configuration function
            config_btn.click(
                fn=setup_alerts,
                inputs=[
                    email_sender,
                    email_password,
                    email_recipient,
                    telegram_token,
                    telegram_chat_id
                ],
                outputs=config_result
            )

        # Tab 3: Test Alerts
        with gr.TabItem("Test Alerts"):
            gr.Markdown("""
            ## Test Alert System
            Send a test alert to verify your configuration is working correctly.
            """)

            # Test message
            test_message = gr.Textbox(
                label="Test Message",
                value="This is a test alert from the violence detection system."
            )

            # Test button
            test_btn = gr.Button("Send Test Alert")
            test_result = gr.Markdown()

            # Connect to test function
            test_btn.click(
                fn=test_alert,
                inputs=test_message,
                outputs=test_result
            )

# Change the detector to use our enhanced class with alerts
detector = ViolenceDetectionSystemWithAlerts(
    violence_path=VIOLENCE_MODEL_PATH,
    confidence_threshold=0.6,
    alert_system=alert_system
)

# Update the process_video_for_gradio function to use our new detector
def process_video_for_gradio(video_file):
    """Process video using the enhanced detector with alerts"""
    if video_file is None:
        return None, "No video uploaded. Please upload a video file."

    try:
        # Create a temporary directory for output
        temp_dir = tempfile.mkdtemp()
        output_path = os.path.join(temp_dir, "processed_video.mp4")

        # Process the video
        result_path, summary = detector.process_video(
            video_path=video_file,
            output_path=output_path,
            skip_frames=2  # Process every other frame for speed
        )

        # Handle errors
        if isinstance(summary, dict) and "error" in summary:
            return None, f"Error: {summary['error']}"

        # Format the summary
        if not os.path.exists(output_path):
            return None, "Error processing video: Output file not created."

        # Create a markdown report
        report = f"""
        ## Violence Detection Results

        ### Statistics
        - Total frames analyzed: {summary['total_frames']}
        - Frames with violence: {summary['violence_frames']} ({summary['violence_percentage']:.2f}%)

        ### Assessment
        {generate_assessment(summary['violence_percentage'])}

        ### Detected Events
        """

        if summary['events']:
            for i, event in enumerate(summary['events']):
                report += f"""
                **Event {i+1}**:
                - Start: {event['start_time']}
                - End: {event['end_time']}
                - Max Probability: {event['max_probability']:.2f}
                """
        else:
            report += "No specific violence events detected."

        # Add alert status to report
        if hasattr(alert_system, 'email_configured') or hasattr(alert_system, 'telegram_configured'):
            report += "\n\n### Alert Status"
            if hasattr(alert_system, 'email_configured') and alert_system.email_configured:
                report += f"\n- Email alerts configured for: {alert_system.recipient_email}"
            if hasattr(alert_system, 'telegram_configured') and alert_system.telegram_configured:
                report += f"\n- Telegram alerts configured for chat ID: {alert_system.telegram_chat_id}"

            if summary['violence_percentage'] > 0 and summary['events']:
                report += "\n- Alerts were sent for detected violence events."
            elif summary['violence_percentage'] > 0:
                report += "\n- No specific violence events triggered alerts."

        return output_path, report

    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        print(f"Error in interface: {e}")
        print(error_details)
        return None, f"Error processing video: {str(e)}"

# Launch with public sharing enabled
app.launch(share=True, debug=True)

Loading YOLOv5 from PyTorch Hub...


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-4-28 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


YOLOv5 model loaded successfully
Loading violence detection model from /content/drive/MyDrive/violence_detector_model (2).onnx...
Violence detection model loaded successfully
Model Input Details:
  Name: input, Shape: ['batch_size', 3, 8, 224, 224], Type: tensor(float)
Violence detection system initialized on cpu
Violence confidence threshold: 0.6
Violence detection system with alerts initialized
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a48cbb8d886d514960.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processing video: /tmp/gradio/0823c513082efea44befb3e837cb27523146798ffab71acbb2648bd808617858/WhatsApp Video 2025-04-28 at 20.12.37_178370fb.mp4
Resolution: 480x854, FPS: 30.0, Total frames: 398


Processing video:   0%|          | 0/398 [00:00<?, ?it/s]

...

Violence probability: 0.1663 (Threshold: 0.6)
Violence probability: 0.3160 (Threshold: 0.6)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Violence probability: 0.1421 (Threshold: 0.6)
Violence probability: 0.8118 (Threshold: 0.6)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Violence probability: 0.1313 (Threshold: 0.6)
Violence probability: 0.8688 (Threshold: 0.6)

Detected 7 violence events
Violence percentage: 52.26%
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a48cbb8d886d514960.gradio.live
